---     
# Creating the `client_assertion` JWT


## **`x5t`**
You need to calculate the **`x5t`** value that will go into the header of the JWT. The completed JWT will be included in the `client_assertion` parameter as it's value. 
> This will be used instead of a `client_secret`.
- Reference: [K000132212: APM: About oAuth x5t (cert-thumbprint-sha1) parameter calculation](https://my.f5.com/manage/s/article/K000132212)

### Calculating **`x5t`**
The value is calculated using the certficate's thumbprint. You can use the command below in your Shell to calculate the **`x5t`**:
```Shell
openssl x509 -noout -in </home/user/pathCert/cert.crt> -fingerprint -sha1 | cut -f2 -d'=' | sed 's/://g' | xxd -r -ps | base64 | sed 's/=//g
```

However, I have provided code below so you can do it all within the notebook.

In [ ]:
import pyperclip
import subprocess

Enter the path for the certificate to calculate the `x5t`:

In [ ]:
cert_path = ''
#cert_path = input('Enter the path to the certificate: ')

This will run the command and capture the calculated **`x5t`**, ready to use in the JWT header.

In [ ]:
command = f"openssl x509 -noout -in {cert_path} -fingerprint -sha1 | cut -f2 -d'=' | sed 's/://g' | xxd -r -ps | base64 | sed 's/=//g'"
result = subprocess.run(command, shell=True, capture_output=True, text=True)

x5t = ''
if result.returncode == 0:
    output = result.stdout.strip()
    print(output)
    x5t = output
else:
    error = result.stderr.strip()
    print(f"Command execution failed with error: {error}")

Provide the tenant id for the application:

In [ ]:
tenant_id = ''
#tenant_id = input('Enter the tenant id: ')

The following URL will be used for the audience (`aud`) claim in the token.

In [ ]:
token_endpoint = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token'
pyperclip.copy(token_endpoint)
print(token_endpoint)
aud = token_endpoint

Now we need the App ID/`client_id`, which will be used for both the issuer (`iss`) and subject (`sub`) claims in the token.

In [ ]:
#client_id = input('Enter the client id: ')
client_id = ''
pyperclip.copy(client_id)
iss = client_id
sub = client_id

Enter the path for th certificate's private key, the key will be used to sign this JWT.

In [ ]:
#private_key_path = input('Enter the path to the private key: ')
private_key_path = ''

Now we will create the  **`client_assertion`** JWT:

In [ ]:
import sys
sys.path.append('../BST/JWT_Utils')
from jwt_sign import sign_jwt


token = sign_jwt(private_key_path, x5t, aud, iss, sub) 
#token = sign_jwt('/path/to/private/key', x5t='your_x5t', aud='your_aud', iss='your_iss', sub='your_sub') #Alternative way to call the function
pyperclip.copy(token)
print(f'JWT Token: {token}')

> Next step is to take this JWT and use it as the value of the **`client_assertion`** query string parameter in the flow.